In [2]:
import scipy.io
import numpy as np
import itertools
from scipy.signal import butter, lfilter
from scipy.fftpack import fft, fftfreq
from random import randint
import scipy.signal
import pandas as pd
import xarray as xr
from pprint import pprint
from pandas import ExcelWriter
from pandas import ExcelFile
from functools import reduce
import re
from matplotlib import pyplot as plt
from IPython.display import display, Markdown
import math
import pylab
import operator
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.cluster import KMeans
from sklearn import metrics 
from sklearn.svm import SVC 
from sklearn.metrics import precision_recall_fscore_support
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import xlwt
from xlrd import open_workbook
from openpyxl import load_workbook
import xlrd
import xlutils
from xlutils.copy import copy
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')

## DATA pre-processing

In [3]:
#import datasets #Ignoring Condition 1 index0
X_im = np.load('Dataset_10_subjs_60secs_5conds_filrange6and25Hz.npy')
X_im_add = np.load('Dataset_Additional_6_subjs_60secs_5conds_filrange6and25Hz.npy')
print("Dimesion of dataset: ", X_im.shape)
print("Dimesion of dataset_add: ", X_im_add.shape)
print("'''''Import successfully'''''")

Dimesion of dataset:  (5, 10, 15000)
Dimesion of dataset_add:  (5, 6, 15000)
'''''Import successfully'''''


In [3]:
#Combine datasets
X_import = np.concatenate((X_im, X_im_add), axis=1)
print("Dimesion of datasets is: ", X_import.shape)

Dimesion of datasets is:  (5, 16, 15000)


In [4]:
# Characteristic of the data
num_cond = int(X_import.shape[0]) 
person_num = int(X_import.shape[1])
smp_freq = 250  # 250 Hz
win_len = 3 # 3s  Using sliding window size 3 secs

In [5]:
# sliding window by using size of 3 seconds
len_data = int(X_import.shape[2]/smp_freq) #Find new length data
number_window = int(len_data-win_len+1) 

X_tr = np.ones((num_cond, person_num, number_window, win_len*smp_freq))

for cond in range(num_cond):
    for p in range(person_num): #number of prople
        for idx in range(number_window): #num of slice
            X_tr[cond, p, idx, :] = X_import[cond, p, idx*smp_freq:(idx+ win_len) * smp_freq]

print("Dimension of data after used sliding window:", X_tr.shape)

print("'''''Applied Sliding Window Successfully'''''")


Dimension of data after used sliding window: (5, 16, 58, 750)
'''''Applied Sliding Window Successfully'''''


In [6]:
# # sliding window by using size of 1 second
# len_data_point = X_import.shape[2]
# overlap = 0.5 #50 percentages
# win_len_point = int(win_len*smp_freq)
# number_window = int(((len_data_point-win_len_point)/(win_len_point*overlap))+1)

# X_tr = np.ones((num_cond, person_num, number_window, win_len*smp_freq))

# for cond in range(num_cond):
#     for p in range(person_num): #number of prople
#         for idx in range(number_window): #num of slice
#             start_pos = int(idx * win_len_point * overlap)
#             stop_pos = int(start_pos + win_len_point)
#             X_tr[cond, p, idx, :] = X_import[cond, p, start_pos:stop_pos]


# print("Dimension of data after used sliding window:", X_tr.shape)

# print("'''''Applied Sliding Window Successfully'''''")

In [7]:
def ITR_calculator(accuracy, num_class, win_len, step, num_sliding):
# Input:
#   n   : # of targets
#   p   : Target identification accuracy (0 <= p <= 1) 
#   t   : Averaged time for a selection [s]

# Output:
#   itr : Information transfer rate [bits/min]

# Reference:
#   [1] M. Cheng, X. Gao, S. Gao, and D. Xu,
#       "Design and Implementation of a Brain-Computer Interface With High 
#        Transfer Rates",
#       IEEE Trans. Biomed. Eng. 49, 1181-1186, 2002.
    average_time = (win_len + (num_sliding-1)*step)/num_sliding
    M = num_class
    P = accuracy
    T = average_time # 0.55 or 2secs for gaze shifting times
    if P < 0 and 1 < P:
        print('stats_itr_BadInputValue_Accuracy need to be between 0 and 1')
    elif P < 1/M:
        print('stats:itr:BadInputValue_The ITR might be incorrect because the accuracy < chance level.')
        ITR = 0
    elif P == 1:
        ITR = np.log2(M)*(60/T)
    else:
        help_term = (1-P)/(M-1)
        ITR = (np.log2(M)+(P*np.log2(P))+((1-P)*np.log2(help_term)))*(60/T)
    return ITR

## Using CCA for frequency recognition

## Experiment Constants

In [8]:
# condition = 1       #1-4 ignore con0
PI = np.pi
sampling_frequency = smp_freq
candidate_frequency = [7.5, 10]
reference_signal_phase = 0
correct_frequency = 7.5
num_harmonic = 3

In [9]:
# Generate Reference Frequency
sin = lambda f, h, t, p: np.sin(2*PI*f*h*t + p)
cos = lambda f, h, t, p: np.cos(2*PI*f*h*t + p)
ref_wave = lambda f, h, t, p: [sin(f, h, t, p), cos(f, h, t, p)]

def generate_reference_signal_at_time(f, t, max_harmonic, phase):
    values = []
    for h in range(1, max_harmonic + 1):
        values += ref_wave(f, h, t, phase)
    return values

def generate_reference_signal(frequency, sampling_frequency, total_time, max_harmonic, phase):
    ref_signal = []
    num_time_step = total_time * sampling_frequency
    for step in range(num_time_step):
        time = step * 1/sampling_frequency
        ref_signal_at_t = generate_reference_signal_at_time(frequency, time, max_harmonic, phase)
        ref_signal.append(ref_signal_at_t)
    return ref_signal

In [10]:
# Solve for Maximum CCA from two multidimensional signal
def find_maximum_canonical_correlations(X, Y):
    if X.shape[0] == Y.shape[0]:
        N = X.shape[0]
    else:
        print('time frame is not equal')
        return None
    C_xx = 1/N * (X.T @ X)
    C_yy = 1/N * (Y.T @ Y)
    C_xy = 1/N * (X.T @ Y)
    C_yx = 1/N * (Y.T @ X)
    C_xx_inv = np.linalg.pinv(C_xx)
    C_yy_inv = np.linalg.pinv(C_yy)
    eig_values, eig_vectors = scipy.linalg.eig(C_yy_inv @ C_yx @ C_xx_inv @ C_xy)
    sqrt_eig_values = np.sqrt(eig_values)
    return max(sqrt_eig_values)

## Generate Reference Signal

*Generate reference signal from candidate frequency upto 6th harmonic*

## CCA main programs

In [11]:
def classify_cca(input_data, ref_signal):
    input_signal = pd.DataFrame(input_data)
    max_cca = {}
    for ref_signal_key in ref_signal.keys():
        value = find_maximum_canonical_correlations(input_signal, ref_signal[ref_signal_key])
        if value.imag == 0.0:
            value = value.real
        else:
            value = None
        max_cca[ref_signal_key] = value
#     max_cca["result"] = max(max_cca, key=max_cca.get)[0]
#     max_cca["result"] = max(max_cca.items(), key=operator.itemgetter(1))[0]
#     max_cca["result"] = max(max_cca.iterkeys(), key=lambda k: max_cca[k])
    max_cca["result"] = max(max_cca.items(), key = lambda x: x[1])[0]
    return max_cca

In [12]:
for con_cca in range(1, 5): 
    print("Condition_is_holding", con_cca)
    ref_signal = {}
    for frequency in candidate_frequency:
        signal = generate_reference_signal(
                frequency=frequency,
                sampling_frequency=smp_freq,
                total_time=win_len ,
                max_harmonic=num_harmonic,
                phase=reference_signal_phase
        )
        ref_signal[frequency] = pd.DataFrame(signal)
        
    result_cca = {}
    for people_index in range(person_num):
        result_cca[f'person_{people_index}'] = []
        for window_index in range(number_window):
            result_cca[f'person_{people_index}'].append(
                classify_cca(X_tr[con_cca, people_index, window_index], ref_signal)
            )
        pd.DataFrame(result_cca[f'person_{people_index}'])

    #ITR_calculator(accuracy, num_class, gaze_time, win_len, step, num_sliding)
    Acc_cca = []
    ITR_re_cca = []
    for p_cca in range(person_num):
        print("Now is person", p_cca)
        display(Markdown(f'## Person #{p_cca}'))
        columns_cca = (pd.DataFrame(result_cca[f'person_{p_cca}'])).iloc[:,[0,1]].columns.values
        value_cca = (pd.DataFrame(result_cca[f'person_{p_cca}'])).iloc[:,[0,1]].idxmax(axis=1)
        display(Markdown('Should be classify as **7.5Hz**'))
        display(Markdown(f"Correctly classify {np.sum(value_cca == correct_frequency)} / {len(value_cca)}"))
        display(Markdown(f"Accuracy {np.sum(value_cca == correct_frequency) / len(value_cca)}"))
        #Calculate accuracy and itr
        acc_cca = np.sum(value_cca == correct_frequency) / len(value_cca)
        itr_cca = ITR_calculator(acc_cca, 2 , win_len, 0.5, number_window)
        
        display(Markdown(f"ITR {itr_cca}"))
        ITR_re_cca.append(itr_cca)
        Acc_cca.append(np.sum(value_cca == correct_frequency) / len(value_cca))
        display(pd.DataFrame(result_cca[f'person_{p_cca}']))

#         #Collect data in Exel file
#         # Initialize a workbook
#         read_book = open_workbook("Revise Result FBCCA vs CCA Allwin.xls")
#         write_book = copy(read_book)

#         # Add a sheet to the workbook
#         row = write_book.get_sheet(0)

#         #The fill data on table (Row, Col, Val)
#         row.write(p_cca+2, 0, p_cca+1)
#         row.write(p_cca+2, con_cca, acc_cca)
#         row.write(p_cca+2, con_cca+10, itr_cca)

#         write_book.save("Revise Result FBCCA vs CCA Allwin.xls")
        
    display(Markdown(f"Mean Accuracy {sum(Acc_cca)/len(Acc_cca)}"))
    display(Markdown(f"Mean ITR {sum(ITR_re_cca)/len(ITR_re_cca)}"))

Condition_is_holding 1
Now is person 0


## Person #0

Should be classify as **7.5Hz**

Correctly classify 50 / 58

Accuracy 0.8620689655172413

...

Accuracy 0.9655172413793104

ITR 86.56948174824126

,7.5,10,result
0,0.236083,0.203899,7.5
1,0.330013,0.179891,7.5
2,0.390579,0.285961,7.5
3,0.372241,0.070706,7.5
4,0.243229,0.138980,7.5
5,0.357680,0.299255,7.5
6,0.323310,0.210408,7.5
7,0.227520,0.209465,7.5
8,0.361981,0.214600,7.5
9,0.512968,0.172982,7.5


Mean Accuracy 0.9116379310344827

Mean ITR 71.70291545349926

## FBCCA

## FBCCA constant

In [13]:
# Band Pass Filter Constant
filter_order = 2
subband_width = 8
# add pad frequency to both and high
PASSBAND_RIPPLE = 100 #0.5 is default
padding_frequency = 2
num_subband = 8
# from paper
subband_weight_a = 1.25
subband_weight_b = 0.25

In [14]:
# Band Filter Input Signal
def filter_signal_verson2(data, low_cutoff_frequency, high_cutoff_frequency):
    low = low_cutoff_frequency
    high = high_cutoff_frequency
    rp = PASSBAND_RIPPLE
    # use Chebyshev Type 1 Filter as described in the Filter Bank paper
    # Ref: https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.signal.cheby1.html
    sos = scipy.signal.cheby1(filter_order, rp, [low, high], btype='bandpass', analog=False, fs=sampling_frequency, output='sos')
    # filtfilt has some issued with number precision so we will use
    # SOS (Second-order (biquadratic) IIR digital filtering)
    # view Limitations Section in https://www.mathworks.com/help/signal/ref/cheby1.html
    filtered_signal = scipy.signal.sosfiltfilt(sos, data, axis=0)
    return filtered_signal

In [15]:
def generate_subband_cutoff(subband_type, start_frequency, subband_width, padding_frequency, num_subband):
    # M1
    if subband_type == 'M1':
        return [
            (start_frequency + subband_width * i - padding_frequency,
             start_frequency + subband_width * (i + 1) + padding_frequency)
            for i in range(1,  num_subband + 1)
        ]
    if subband_type == 'M2':
        last_cutoff = start_frequency + subband_width * (num_subband + 1) + padding_frequency
        return [
            (start_frequency + subband_width * i - padding_frequency,
             min(start_frequency + 2 * subband_width * i + padding_frequency, last_cutoff))
            for i in range(1,  num_subband + 1)
        ]
    # M3
    if subband_type == 'M3':
        last_cutoff = start_frequency + subband_width * (num_subband + 1) + padding_frequency
        return [
            (start_frequency + subband_width * i - padding_frequency,
             last_cutoff)
            for i in range(1, num_subband + 1)
        ]

In [16]:
# Sub band signal weight for FBCCA
# optimal a, b from the paper
def subband_weight(n, a=1.25, b=0.25):
    return n ** (-a) + b

In [17]:
def get_subband_weight_vector(num_subband, a=1.25, b=0.25):
    return np.array([subband_weight(n+1, a, b) for n in range(num_subband)])

## FBCCA main program

In [18]:
subbands = generate_subband_cutoff('M3', 0, subband_width, padding_frequency, num_subband)
display(subbands)

[(6, 74), (14, 74), (22, 74), (30, 74), (38, 74), (46, 74), (54, 74), (62, 74)]

In [19]:
def classify_fbcca(data, ref_signal, a, b):
    filtered_signal = {}
    input_signal = data
    for subband in subbands:
        filtered_signal[f"{subband[0]}_{subband[1]}"] = \
            pd.DataFrame(filter_signal_verson2(input_signal, subband[0], subband[1]))
    max_cca = {}

    for subband_key in filtered_signal.keys():
        for ref_signal_key in ref_signal.keys():
            value = find_maximum_canonical_correlations(filtered_signal[subband_key], ref_signal[ref_signal_key])
            if ref_signal_key not in max_cca:
                max_cca[ref_signal_key] = []
            if value.imag == 0.0:
                value = value.real
            if ref_signal_key not in max_cca:
                max_cca[ref_signal_key] = []
            max_cca[ref_signal_key].append(value)
    rho = {}
    for key in max_cca.keys():
        rho[key] = np.array(max_cca[key])
    weight = get_subband_weight_vector(num_subband, a, b)
    result = {}
    for key in rho.keys():
        result[key] = np.sum(weight * rho[key] ** 2)
    result["result"] = max(result.items(), key=operator.itemgetter(1))[0] 
    return result

In [20]:
for con_fbcca in range(1, 5): 
    print("Condition_is_holding", con_fbcca)
    ref_signal_fbcca = {}
    for frequency in candidate_frequency:
        signal_fbcca = generate_reference_signal(
                frequency=frequency,
                sampling_frequency=smp_freq,
                total_time=win_len ,
                max_harmonic=num_harmonic,
                phase=reference_signal_phase
        )
        ref_signal_fbcca[frequency] = pd.DataFrame(signal_fbcca)
    
    result_fbcca = {}
    for people_index2 in range(person_num):
        result_fbcca[f'person_{people_index2}'] = []
        for window_index2 in range(number_window):
            result_fbcca[f'person_{people_index2}'].append(
                classify_fbcca(X_tr[con_fbcca, people_index2, window_index2], ref_signal_fbcca,
                               subband_weight_a, subband_weight_b)
            )
        pd.DataFrame(result_fbcca[f'person_{people_index2}'])

    #ITR_calculator(accuracy, num_class, gaze_time, win_len, step, num_sliding)
    Acc_fbcca = []
    ITR_re_fbcca = []
    for p_fbcca in range(person_num):
        print("Now is person", p_fbcca)
        display(Markdown(f'## Person #{p_fbcca}'))
        columns_fbcca = (pd.DataFrame(result_fbcca[f'person_{p_fbcca}'])).iloc[:,[0,1]].columns.values
        value_fbcca = (pd.DataFrame(result_fbcca[f'person_{p_fbcca}'])).iloc[:,[0,1]].idxmax(axis=1)
        display(Markdown('Should be classify as **7.5Hz**'))
        display(Markdown(f"Correctly classify {np.sum(value_fbcca == correct_frequency)} / {len(value_fbcca)}"))
        display(Markdown(f"Accuracy {np.sum(value_fbcca == correct_frequency) / len(value_fbcca)}"))
        #Calculate accuracy and itr
        acc_fbcca = np.sum(value_fbcca == correct_frequency) / len(value_fbcca)
        itr_fbcca = ITR_calculator(acc_fbcca, 2 , win_len, 0.5, number_window)
        
        display(Markdown(f"ITR {itr_fbcca}"))
        ITR_re_fbcca.append(itr_fbcca)
        Acc_fbcca.append(np.sum(value_fbcca == correct_frequency) / len(value_fbcca))
        display(pd.DataFrame(result_fbcca[f'person_{p_fbcca}']))

#         #Collect data in Exel file
#         # Initialize a workbook
#         read_book = open_workbook("Revise Result FBCCA vs CCA Allwin.xls")
#         write_book = copy(read_book)

#         # Add a sheet to the workbook
#         row = write_book.get_sheet(0)

#         #The fill data on table (Row, Col, Val)

#         row.write(p_fbcca+2, con_fbcca+4, acc_fbcca)
#         row.write(p_fbcca+2, 9, win_len)
#         row.write(p_fbcca+2, 10, p_fbcca+1)
#         row.write(p_fbcca+2, con_fbcca+14, itr_fbcca)
       

#         write_book.save("Revise Result FBCCA vs CCA Allwin.xls")
        
    display(Markdown(f"Mean Accuracy {sum(Acc_fbcca)/len(Acc_fbcca)}"))
    display(Markdown(f"Mean ITR {sum(ITR_re_fbcca)/len(ITR_re_fbcca)}"))

Condition_is_holding 1
Now is person 0


## Person #0

Should be classify as **7.5Hz**

Correctly classify 57 / 58

Accuracy 0.9827586206896551

...

Accuracy 1.0

ITR 110.47619047619047

,7.5,10,result
0,0.031476,0.003213,7.5
1,0.036785,0.003636,7.5
2,0.056955,0.005397,7.5
3,0.037215,0.003194,7.5
4,0.018654,0.002794,7.5
5,0.036581,0.004651,7.5
6,0.044487,0.003688,7.5
7,0.018973,0.003011,7.5
8,0.044809,0.003399,7.5
9,0.020531,0.003657,7.5


Mean Accuracy 0.9978448275862069

Mean ITR 108.74091264047402

In [ ]:
print("Done")